# Exercise 2

Hi, In this exercise we are going to build an Imputation pipeline with the help of the knowledge we just gained in lecture 2 and to make this happen we will use a car price prediction problem dataset. Lets Impute!!

In [ ]:
pip install feature-engine
#Run this cell  before any other cell

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from feature_engine.imputation import MeanMedianImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
#Function to Load Data 
def getData():
    cols=['symboling','stroke','compressionratio','citympg','enginelocation','aspiration']
    df=pd.read_csv('https://raw.githubusercontent.com/univai-ghf/FeatureProcessingAndSelectionWorkshop/main/Data/Nandata1.csv',usecols=cols)
    return df


In [4]:
#Load your data here 
data=getData()
data.head()

,symboling,aspiration,enginelocation,stroke,compressionratio,citympg
0,3.0,std,front,2.68,9.0,21.0
1,3.0,std,front,2.68,9.0,21.0
2,1.0,std,front,3.47,NaN,19.0
3,2.0,std,front,3.40,10.0,24.0
4,NaN,std,NaN,3.40,8.0,NaN


In [7]:
#1. Calculate the percentage null values in our dataset.
def calcNull(data:pd.DataFrame)->pd.DataFrame:
    '''Ths function returns null value information in the dataset. A dataframe containing the results is returned.
    
       data : The dataset containing null values

    '''
    null=data.isnull().sum()
    nullData=pd.DataFrame()
    nullData['features']=null.index
    nullData['noOfNullValues']=null.values
    nullData['percNullValues']=np.round((nullData['noOfNullValues']/len(data))*100,2)
    return nullData

calcNull(data)

,features,noOfNullValues,percNullValues
0,symboling,37,18.05
1,aspiration,20,9.76
2,enginelocation,20,9.76
3,stroke,20,9.76
4,compressionratio,37,18.05
5,citympg,20,9.76


In [10]:
#2. Check the cardinality of the data 
print(data.nunique())

symboling            6
aspiration           2
enginelocation       2
stroke              35
compressionratio    30
citympg             28
dtype: int64


In [18]:
#3. On the basis of the above unique values choose categorical variable and Continious variables
categorical=['enginelocation','aspiration']
continious=['symboling','stroke','compressionratio','citympg']

#If you look carefully symboling is an ordinal variable, we just included it in continious for the sake of the example.

In [19]:
#4. Setup A pipeline for imputing Categorical Variables, Use most_frequent Stratergy for filling in the missing values
categoricalPipeline=Pipeline([
    ('categorical_Imputer',SimpleImputer(strategy='most_frequent'))
]) 

In [30]:
#5 Set up a pipeline for imputing continious variables, you may use mean meadian or most_frequent stratergy.
numericalPipeline =Pipeline([
    ('numericalImputer',MeanMedianImputer(variables=continious))
])

In [33]:
#6 Integrate both the pipelines so they can act on the dataset together. Hint: Use ColumnTransformer for integrating two pipelines
imputingPipeline=ColumnTransformer([
    ('categoricalPipeline',categoricalPipeline,categorical),
    ('numericalPipeline',numericalPipeline,continious)
])


In [36]:
#7 Fit the pipeline on your dataset
imputingPipeline.fit(data)

ColumnTransformer(transformers=[('categoricalPipeline',
                                 Pipeline(steps=[('categorical_Imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['enginelocation', 'aspiration']),
                                ('numericalPipeline',
                                 Pipeline(steps=[('numericalImputer',
                                                  MeanMedianImputer(variables=['symboling',
                                                                               'stroke',
                                                                               'compressionratio',
                                                                               'citympg']))]),
                                 ['symboling', 'stroke', 'compressionratio',
                                  'citympg'])])

In [38]:
#8. Use .transform method of the imputingPipeline and save the resulting dataset in a a variable X and print X
X=imputingPipeline.transform(data)
print(X)

[['front' 'std' 3.0 2.68 9.0 21.0]
 ['front' 'std' 3.0 2.68 9.0 21.0]
 ['front' 'std' 1.0 3.47 9.0 19.0]
 ...
 ['front' 'std' -1.0 2.87 8.8 18.0]
 ['front' 'turbo' -1.0 3.4 23.0 26.0]
 ['front' 'turbo' -1.0 3.15 9.5 19.0]]


In [40]:
#9. Create a dataframe of X and check for any null values
XDf=pd.DataFrame(X)
calcNull(XDf)

,features,noOfNullValues,percNullValues
0,0,0,0.0
1,1,0,0.0
2,2,0,0.0
3,3,0,0.0
4,4,0,0.0
5,5,0,0.0
